# Qualitative analysis

# Read data

In [1]:
#@title Download all
!wget https://raw.githubusercontent.com/OpenBioLink/ITO/master/notebooks/simon_trajectories_notebooks/all.csv
!wget https://raw.githubusercontent.com/OpenBioLink/ITO/master/notebooks/simon_trajectories_notebooks/polarities.csv
!wget https://raw.githubusercontent.com/OpenBioLink/ITO/master/notebooks/simon_trajectories_notebooks/trajectory_grouping_ITO_00101.csv
!wget https://raw.githubusercontent.com/OpenBioLink/ITO/master/notebooks/simon_trajectories_notebooks/trajectory_grouping_ITO_00141.csv

--2022-01-25 12:52:46--  https://raw.githubusercontent.com/OpenBioLink/ITO/master/notebooks/simon_trajectories_notebooks/all.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21395920 (20M) [text/plain]
Saving to: ‘all.csv’

all.csv             100%[===================>]  20.40M  --.-KB/s    in 0.07s   

2022-01-25 12:52:47 (306 MB/s) - ‘all.csv’ saved [21395920/21395920]

--2022-01-25 12:52:47--  https://raw.githubusercontent.com/OpenBioLink/ITO/master/notebooks/simon_trajectories_notebooks/polarities.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting r

In [5]:
#@title Dependencies
!pip install plotly==5.5.0

In [3]:
#@title Read all necessary data
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

YEAR_NOW = 2021

polarities = pd.read_csv("polarities.csv", sep=";")

all = pd.read_csv("all.csv", parse_dates=["date"])
all["result"] = all["result"].astype("float")

# Warning: run following lines only ONCE after reading all.csv (Otherwise polarities are changed again)
# invert all negative polarity metrics
all.loc[all["metric_label"].isin(polarities[polarities["polarity"] == "neg"]["metric"]), "result"] *= -1
# filter all datasets that have leq 3 entries
# all = all.groupby(["top_level_class_label", "task_label", "dataset_label", "metric_label"]).filter(lambda x: len(x) > 3).reset_index()
# keep all entries with top level class nlp or cv
all = all[(all["top_level_class"] == "https://identifiers.org/ito:ITO_00101") | (all["top_level_class"] == "https://identifiers.org/ito:ITO_00141")]
all["year"] = all["date"].dt.year
all.head(5)

,date,top_level_class,top_level_class_label,task_label,dataset,dataset_label,paper,paper_label,model,model_label,metric,metric_label,result,year
2,2004-07-01,https://identifiers.org/ito:ITO_00141,Natural language processing,Unsupervised Dependency Parsing,https://identifiers.org/ito:ITO_30848,Penn Treebank,https://identifiers.org/ito:ITO_30858,Corpus-Based Induction of Syntactic Structure:...,https://identifiers.org/ito:ITO_iAIZzoiwy3CPQwKP7,DMV model in \'Corpus-Based Induction of Synta...,https://identifiers.org/ito:ITO_11157,UAS,35.90,2004
5,2006-05-12,https://identifiers.org/ito:ITO_00101,Vision process,BIRL: Benchmark on Image Registration methods ...,https://identifiers.org/ito:ITO_38642,CIMA-10k,https://identifiers.org/ito:ITO_15836,Consistent and elastic registration of histolo...,https://identifiers.org/ito:ITO_igTBS6Rq9JLhl6K7a,bUnwarpJ model in \'Consistent and elastic reg...,https://identifiers.org/ito:ITO_15829,AMrTRE,-2.82,2006
7,2006-05-12,https://identifiers.org/ito:ITO_00101,Vision process,BIRL: Benchmark on Image Registration methods ...,https://identifiers.org/ito:ITO_38642,CIMA-10k,https://identifiers.org/ito:ITO_15836,Consistent and elastic registration of histolo...,https://identifiers.org/ito:ITO_igTBS6Rq9JLhl6K7a,bUnwarpJ model in \'Consistent and elastic reg...,https://identifiers.org/ito:ITO_15830,MMrTRE,3.00,2006
9,2006-06-01,https://identifiers.org/ito:ITO_00141,Natural language processing,Constituency Parsing,https://identifiers.org/ito:ITO_34534,Penn Treebank,https://identifiers.org/ito:ITO_34554,Effective Self-Training for Parsing,https://identifiers.org/ito:ITO_iuKB4ddIY2bq0Eq1Q,Self-training model in \'Effective Self-Traini...,https://identifiers.org/ito:ITO_03847,F1 score,92.10,2006
10,2007-12-01,https://identifiers.org/ito:ITO_00101,Vision process,Image Classification,https://identifiers.org/ito:ITO_37799,STL-10,https://identifiers.org/ito:ITO_37887,Receptive Fields without Spike-Triggering,https://identifiers.org/ito:ITO_i57NAEhGogw935KM4,Receptive Fields model in \'Receptive Fields w...,https://identifiers.org/ito:ITO_08679,Percentage correct,60.10,2007


# Seperate charts

## New Benchmarks

Below chart depicts the number of new benchmarks every year (Different metrics of a benchmark are aggregated, meaning the year a benchmark is reported as NEW is the year **any** metric is first reported)

\# results > 3: benchmark has at least 3 reported results in at least one metric; examples: 
+ True if 4 results in accuracy, only 1 in recall
+ False if 2 results in accuracy, 2 in recall

In [6]:
#@title New benchmarks
all["year"] = all["date"].dt.year

new = all.copy(deep = True)

new["# results > 3"] = True
idxe = all.groupby(["top_level_class_label", "task_label", "dataset_label", "metric_label"]).filter(lambda x: len(x) <= 3).index
new.loc[list(idxe),"# results > 3"] = False

new = new.groupby(["top_level_class", "top_level_class_label", "task_label", "dataset_label", "year"])["# results > 3"].any().reset_index()

new = new.groupby(["top_level_class", "top_level_class_label", "task_label", "dataset_label", "# results > 3"])['year'].min().reset_index()

new = new[new["year"] < YEAR_NOW]


for tlc, grp in new.groupby(["top_level_class_label"]):   
    # group = pd.read_csv(f"trajectory_grouping_{grp.iloc[0]['top_level_class'].replace('https://identifiers.org/ito:', '')}.csv")
    grp["year"] = grp["year"].astype(str)
    grp = grp.sort_values(by="# results > 3", ascending=False)
    # set cumulative = True, # benchmarkes created till now
    fig = px.histogram(grp, x="year", title=f"New benchmarks: {tlc}", color="# results > 3").update_xaxes(categoryorder='category ascending')
    fig.show()

## Disbanded benchmarks

Below chart depicts the number of disbanded benchmarks every year (Different metrics of a benchmark are aggregated, meaning the year a benchmark is reported as disbanded is the year **after** **any** metric is last reported)

Important **year after**

\# results > 3: benchmark has at least 3 reported results in at least one metric; examples: 
+ True if 4 results in accuracy, only 1 in recall
+ False if 2 results in accuracy, 2 in recall

In [7]:
#@title Disbanded benchmarks
disbanded = all.copy(deep = True)

disbanded["# results > 3"] = True
idxe = all.groupby(["top_level_class_label", "task_label", "dataset_label", "metric_label"]).filter(lambda x: len(x) <= 3).index
disbanded.loc[list(idxe),"# results > 3"] = False

disbanded = disbanded.groupby(["top_level_class", "top_level_class_label", "task_label", "dataset_label", "year"])["# results > 3"].any().reset_index()

disbanded = disbanded.groupby(["top_level_class", "top_level_class_label", "task_label", "dataset_label", "# results > 3"])['year'].max().reset_index()
disbanded["year"] = disbanded["year"] + 1

disbanded = disbanded[disbanded["year"] < YEAR_NOW]

for tlc, grp in disbanded.groupby(["top_level_class_label"]):   
    # group = pd.read_csv(f"trajectory_grouping_{grp.iloc[0]['top_level_class'].replace('https://identifiers.org/ito:', '')}.csv")
    grp["year"] = grp["year"].astype(str)
    grp = grp.sort_values(by="# results > 3", ascending=False)
    fig = px.histogram(grp, x="year", title=f'Disbanded benchmarks: {tlc}', color="# results > 3").update_xaxes(categoryorder='category ascending')
    fig.show()

## SOTA analysis

Below chart depicts the number of benchmarks where SOTA results and non-SOTA results were reported. Note: This is not the number of SOTA or non-SOTA results reported every year, it is the number of benchmarks reporting SOTA and non-SOTA results. 

(Different metrics of a benchmark are aggregated, meaning the year a benchmark reports a SOTA is the year **any** metric is a SOTA)


\# results > 3: benchmark has at least 3 reported results in at least one metric; examples: 
+ True if 4 results in accuracy, only 1 in recall
+ False if 2 results in accuracy, 2 in recall

Note: First entries in benchmarks are not considered in this chart (neither sota nor non-sota, they are counted as "New benchmarks", see chart above)

In [8]:
#@title SOTA analysis

def is_sota(x):
    a = x.copy(deep=True)
    a["result"] = a["result"].cummax()
    a["result"] = a.duplicated(subset=["result"]) 
    # uncomment to count first appearance as non-sota
    # a.iloc[0,1] = True
    # count first appearance neither as sota or non-sota
    a = a.iloc[1:, :]
    a = a.set_index(["year"])
    return ~a

# new aggregation (count benchmarks only once)

sota = all.copy(deep = True)

sota["# results > 3"] = True
idxe = all.groupby(["top_level_class_label", "task_label", "dataset_label", "metric_label"]).filter(lambda x: len(x) <= 3).index
sota.loc[list(idxe),"# results > 3"] = False
idxe = sota.groupby(["top_level_class_label", "task_label", "dataset_label"])["# results > 3"].filter(lambda x: x.any()).index
sota.loc[list(idxe),"# results > 3"] = True

#remove
# sota = sota[sota["# results > 3"] == True]

# get the maximum of each year
sota = sota.groupby(["top_level_class", "top_level_class_label", "task_label", "dataset_label", "metric_label", "# results > 3", "year"])['result'].max().reset_index()
# get the sotas of each year
sota = sota.groupby(["top_level_class", "top_level_class_label", "task_label", "dataset_label", "metric_label", "# results > 3"])[["year", "result"]].apply(is_sota).reset_index()
# aggregate sotas over year (at least one sota in any metric)
sota = sota.groupby(["top_level_class", "top_level_class_label", "task_label", "dataset_label", "# results > 3", "year"])["result"].any().reset_index()

sota = sota[sota["year"] < YEAR_NOW]


for tlc, grp in sota.groupby(["top_level_class_label"]):   
    grp["Is SOTA and bench > 3 results"] = grp[["result", "# results > 3"]].apply(lambda x: "SOTA and # results > 3" if (x["result"] and x["# results > 3"]) else "SOTA and not # results > 3" if (x["result"] and not x["# results > 3"]) else "not SOTA and # results > 3" if (not x["result"] and x["# results > 3"]) else "not SOTA and not # results > 3", axis=1)

    # group = pd.read_csv(f"trajectory_grouping_{grp.iloc[0]['top_level_class'].replace('https://identifiers.org/ito:', '')}.csv")
    grp["year"] = grp["year"].astype(str)
    grp = grp.sort_values(by="Is SOTA and bench > 3 results", ascending=False)
    fig = px.histogram(grp, x="year", title=f'SOTA: {tlc}', color="Is SOTA and bench > 3 results").update_xaxes(categoryorder='category ascending')
    fig.show()

# Composite charts

In [9]:
#@title Stacked chart
#@markdown Stacked chart of the above charts

new["result"] = 0
disbanded["result"] = 3
sota.loc[sota["result"] == True, "result"] = 1
sota.loc[sota["result"] == False, "result"] = 2
all_res = pd.concat([new, sota, disbanded])

for tlc, grp in all_res.groupby(["top_level_class_label"]):
    grp["year"] = grp["year"].astype(str)
    grp["result_label"] = grp["result"].map({0: 'New benchmarks', 1: 'New sota', 2: 'New entries but not sota', 3: 'Disbanded benchmarks'})
    # Here we use a column with categorical data
    fig = px.histogram(grp, x="year", color="result_label", pattern_shape="# results > 3", pattern_shape_map={
             True: "", False: "x",
             }, title=f"{tlc}").update_xaxes(categoryorder='category ascending')
    fig.show()

In [10]:
#@title # Active benchmarks
#@markdown Cummulative sum of new - disbanded

# piktogram = all_res[all_res["top_level_class_label"] == "Natural language processing"]
for tlc, piktogram in all_res.groupby("top_level_class_label"):
    piktogram["count"] = 1
    piktogram=piktogram.groupby(["year", "result"])["count"].sum().reset_index()
    piktogram=piktogram[piktogram["year"] < 2020]
    type0 = piktogram[piktogram["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type1 = piktogram[piktogram["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type3 = piktogram[piktogram["result"] == 3][["year", "count"]].sort_values(by="year").set_index("year")

    active = type0.subtract(type3, fill_value=0).cumsum().reset_index()
    sota__ = type1.reset_index()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
                            x=[active["year"].min(), active["year"].max()], y=np.full(2, 0),
                            mode='lines',
                            line_color='white', showlegend=False))

    fig.add_trace(go.Scatter(x=active["year"], y=active["count"],
                    mode='lines', fill='tonexty', line_shape="spline"))

    """
    fig.add_trace(go.Scatter(
                            x=[sota__["year"].min(), sota__["year"].max()], y=np.full(2, 0),
                            mode='lines',
                            line_color='white', showlegend=False))

    fig.add_trace(go.Scatter(x=sota__["year"], y=sota__["count"],
                    mode='lines', fill='tonexty', line_shape="spline"))
    """
        
    #fig.update_traces(orientation='h', side='positive', width=3, points=False)
    fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False, showlegend=False, title=f"{tlc}: # active benchmarks") # that way you hide the y axis ticks labels)
    fig.show()

In [21]:
#@title Piktogram
#@markdown (Legend is interactive; singleclick to toggle on/off certain traces, doubleclick to isolate the trace; this is on every chart btw)

import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np

for tlc, piktogram in all_res.groupby("top_level_class_label"):
    piktogram = piktogram[piktogram["# results > 3"] == True]

    grouping = pd.read_csv(f"trajectory_grouping_{piktogram.iloc[0]['top_level_class'].replace('https://identifiers.org/ito:', '')}.csv")
    grouping.rename(columns={"Class_Label": "task_label"}, inplace=True)

    piktogram = piktogram.merge(grouping, on="task_label", how="left")
    piktogram["task_label"] = piktogram["Suggested_label"]+": "+ piktogram["task_label"]
    piktogram = piktogram.dropna()

    piktogram = piktogram.sort_values(by=["task_label", "year", "result"])

    piktogram["count"] = 1
    piktogram=piktogram.groupby(["task_label", "year", "result"])["count"].sum().reset_index()

    piktogram["result_label"] = piktogram["result"].map({0: 'New benchmarks', 1: 'New sota', 2: 'New entries but not sota', 3: 'Disbanded benchmarks'})

    import plotly.express as px
    fig = px.scatter(piktogram, x="year", y="task_label", color="result_label", symbol="result_label", size='count', height=50*len(piktogram["task_label"].unique()), title=f"{tlc}").update_yaxes(categoryorder='category descending')
    fig.show()

## Gescheiterte Versuche

In [22]:
#@title num active per task NORMALIZED
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

def min_max(x):
    a = x.copy(deep=True)
    a["count"] = (a["count"]) / (a["count"].max())
    a = a.set_index(["year"])
    return a

def cumsum(x):
    a = x.copy(deep=True)
    a = a.sort_values(by=["year"])
    a["count"] = a["count"].cumsum()
    return a.set_index("year")
    
def compose(df):

    a = df.copy(deep=True)
    a = a.drop_duplicates(subset=["year"])
    # set to arbitrary result for now

    type0 = df[df["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type1 = df[df["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type3 = df[df["result"] == 3][["year", "count"]].sort_values(by="year").set_index("year")
    active = type0.subtract(type3, fill_value=0).cumsum()

    sota_prop = type1.divide(active, fill_value=0)

    return active



for tlc, piktogram in all_res.groupby("top_level_class_label"):

  # piktogram = piktogram[piktogram["# results > 3"] == True]

  grouping = pd.read_csv(f"trajectory_grouping_{piktogram.iloc[0]['top_level_class'].replace('https://identifiers.org/ito:', '')}.csv")
  grouping.rename(columns={"Class_Label": "task_label"}, inplace=True)

  piktogram = piktogram.merge(grouping, on="task_label", how="left")
  piktogram["task_label"] = piktogram["Suggested_label"]+": "+ piktogram["task_label"]
  piktogram = piktogram.dropna()

  piktogram = piktogram.sort_values(by=["task_label", "year", "result"])

  piktogram["result_label"] = piktogram["result"].map({0: 'New benchmarks', 1: 'New sota', 2: 'New entries but not sota', 3: 'Disbanded benchmarks'})

  piktogram["count"] = 1
  piktogram=piktogram.groupby(["task_label", "year", "result"])["count"].sum().reset_index()

  # piktogram=piktogram.groupby(["task_label", "result"])["year", "count"].apply(cumsum).reset_index()

  piktogram=piktogram.groupby(["task_label"])["result", "year", "count"].apply(compose).reset_index()

  piktogram=piktogram.groupby(["task_label"])["year", "count"].apply(min_max).reset_index()

  piktogram = piktogram.groupby(["task_label"]).filter(lambda x: len(x) > 1)

  data = [(task, grp) for (task, grp) in piktogram.groupby(["task_label"])]

  colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', len(data), colortype='rgb')

  fig = go.Figure()
  for index, (data_, color) in enumerate(zip(data, colors)):
      fig.add_trace(go.Scatter(
                              x=[data_[1]["year"].min(), data_[1]["year"].max()], y=np.full(2, len(data)-index),
                              mode='lines',
                              line_color='white', showlegend=False))

      fig.add_trace(go.Scatter(x=data_[1]["year"], y=data_[1]["count"] + (len(data)-index) + 0.4,
                      mode='lines',
                      name=data_[0], fill='tonexty', line_shape="spline"))

      fig.add_trace(go.Scatter(
          x=[2010, 2021], y=np.full(2, len(data)-index),
          mode='lines',
          line_color='white', showlegend=False))

      # plotly.graph_objects' way of adding text to a figure
      fig.add_annotation(
          x=2010,
          y=len(data)-index,
          text=f'{data_[0]}',
          showarrow=False,
          yshift=10)
      
  #fig.update_traces(orientation='h', side='positive', width=3, points=False)
  fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False, height=50*len(piktogram["task_label"].unique()), title=f"{tlc}", showlegend=False, yaxis=dict(showticklabels=False)) # that way you hide the y axis ticks labels)
  fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [26]:
#@title num active per task NOT NORMALIZED
#@markdown to get a nice looking chart tasks were filtered heavier than previous chart
#@markdown - Only Benchmarks that have more than 3 results in at least one metric
#@markdown - Only Benchmarks that have more than 1 active benchmarks in at least one year
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

def min_max(x):
    a = x.copy(deep=True)
    a["count"] = (a["count"])
    a = a.set_index(["year"])
    return a

def cumsum(x):
    a = x.copy(deep=True)
    a = a.sort_values(by=["year"])
    a["count"] = a["count"].cumsum()
    return a.set_index("year")
    
def compose(df):

    a = df.copy(deep=True)
    a = a.drop_duplicates(subset=["year"])
    # set to arbitrary result for now

    type0 = df[df["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type1 = df[df["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type3 = df[df["result"] == 3][["year", "count"]].sort_values(by="year").set_index("year")
    active = type0.subtract(type3, fill_value=0).cumsum()

    sota_prop = type1.divide(active, fill_value=0)

    return active



for tlc, piktogram in all_res.groupby("top_level_class_label"):

  piktogram = piktogram[piktogram["# results > 3"] == True]

  grouping = pd.read_csv(f"trajectory_grouping_{piktogram.iloc[0]['top_level_class'].replace('https://identifiers.org/ito:', '')}.csv")
  grouping.rename(columns={"Class_Label": "task_label"}, inplace=True)

  piktogram = piktogram.merge(grouping, on="task_label", how="left")
  piktogram["task_label"] = piktogram["Suggested_label"]+": "+ piktogram["task_label"]
  piktogram = piktogram.dropna()

  piktogram = piktogram.sort_values(by=["task_label", "year", "result"])

  piktogram["result_label"] = piktogram["result"].map({0: 'New benchmarks', 1: 'New sota', 2: 'New entries but not sota', 3: 'Disbanded benchmarks'})

  piktogram["count"] = 1
  piktogram=piktogram.groupby(["task_label", "year", "result"])["count"].sum().reset_index()

  # piktogram=piktogram.groupby(["task_label", "result"])["year", "count"].apply(cumsum).reset_index()

  piktogram=piktogram.groupby(["task_label"])["result", "year", "count"].apply(compose).reset_index()

  piktogram=piktogram.groupby(["task_label"])["year", "count"].apply(min_max).reset_index()

  piktogram = piktogram.groupby(["task_label"]).filter(lambda x: len(x) > 1 and x["count"].max() > 1)

  data = [(task, grp) for (task, grp) in piktogram.groupby(["task_label"])]

  colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', len(data), colortype='rgb')

  fig = go.Figure()
  cur_max = 0
  for index, (data_, color) in enumerate(zip(data, colors)):
      cur_max -= data_[1]["count"].max()
      fig.add_trace(go.Scatter(
                              x=[data_[1]["year"].min(), data_[1]["year"].max()], y=np.full(2, cur_max),
                              mode='lines',
                              line_color='white', showlegend=False))

      fig.add_trace(go.Scatter(x=data_[1]["year"], y=data_[1]["count"] + cur_max + 1,
                      mode='lines',
                      name=data_[0], line_shape="spline", fill='tonexty'))

      fig.add_trace(go.Scatter(
          x=[2010, 2021], y=np.full(2, cur_max),
          mode='lines',
          line_color='white', showlegend=False))

      
      # plotly.graph_objects' way of adding text to a figure
      fig.add_annotation(
          x=2010,
          y=cur_max,
          text=f'{data_[0]}',
          showarrow=False,
          yshift=10)
      
  #fig.update_traces(orientation='h', side='positive', width=3, points=False)
  fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False, height=80*len(piktogram["task_label"].unique()), title=f"{tlc}", showlegend=False, yaxis=dict(showticklabels=False)) # that way you hide the y axis ticks labels)
  fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [27]:
#@title Comparison number sota <-> active

import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

def min_max(x):
    a = x.copy(deep=True)
    a["count"] = (a["count"]) / (a["count"].max())
    a = a.set_index(["year"])
    return a
    
def compose(df):
    # set to arbitrary result for now

    type0 = df[df["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type1 = df[df["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type3 = df[df["result"] == 3][["year", "count"]].sort_values(by="year").set_index("year")
    active = type0.subtract(type3, fill_value=0).cumsum()

    sota_prop = type1.divide(active, fill_value=0)

    print(type0.cumsum())
    print(type3.cumsum())
    print(active)
    print(type1)

    return sota_prop.reset_index()



for tlc, piktogram in all_res.groupby("top_level_class_label"):

  # piktogram = piktogram[piktogram["# results > 3"] == True]

  """
  grouping = pd.read_csv("trajectory_grouping_ITO_00141.csv")
  grouping.rename(columns={"Class_Label": "task_label"}, inplace=True)

  piktogram = piktogram.merge(grouping, on="task_label", how="left")
  piktogram["task_label"] = piktogram["Suggested_label"]+": "+ piktogram["task_label"]
  piktogram = piktogram.dropna()
  """

  piktogram = piktogram.sort_values(by=["task_label", "year", "result"])

  piktogram["result_label"] = piktogram["result"].map({0: 'New benchmarks', 1: 'New sota', 2: 'New entries but not sota', 3: 'Disbanded benchmarks'})

  piktogram["count"] = 1
  piktogram=piktogram.groupby(["year", "result"])["count"].sum().reset_index()

  # piktogram=piktogram.groupby(["task_label", "result"])["year", "count"].apply(cumsum).reset_index()


  # Compose

  type0 = piktogram[piktogram["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
  type1 = piktogram[piktogram["result"] == 1][["year", "count"]].sort_values(by="year").set_index("year")
  type3 = piktogram[piktogram["result"] == 3][["year", "count"]].sort_values(by="year").set_index("year")

  active = type0.subtract(type3, fill_value=0).cumsum()
  active = active.reindex(range(active.index.min(),active.index.max() + 1)).ffill()

  # active_norm = min_max(active.reset_index()).reset_index()
  active_norm = active.reset_index()

  # type1 = type1.reindex(range(active.index.min(),active.index.max())).interpolate()
  # sota_prop = type1.divide(active, fill_value=0).reset_index()
  sota_prop = type1.reset_index()

  """
  print(type0.cumsum())
  print(type3.cumsum())
  print(active)
  print(type1)
  print(sota_prop)
  """

  fig = go.Figure()

  fig.add_trace(go.Scatter(
                          x=[sota_prop["year"].min(), sota_prop["year"].max()], y=np.full(2, 0),
                          mode='lines',
                          line_color='white', showlegend=False))

  fig.add_trace(go.Scatter(x=sota_prop["year"], y=sota_prop["count"],
                  mode='lines', fill='tonexty', line_shape="spline"))

  fig.add_trace(go.Scatter(
                          x=[active_norm["year"].min(), active_norm["year"].max()], y=np.full(2, 0),
                          mode='lines',
                          line_color='white', showlegend=False))

  fig.add_trace(go.Scatter(x=active_norm["year"], y=active_norm["count"],
                  mode='lines', fill='tonexty', line_shape="spline"))
      
  #fig.update_traces(orientation='h', side='positive', width=3, points=False)
  # yaxis=dict(showticklabels=False)
  fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False, height=1000, showlegend=False, 
      title=f"{tlc}: red - #SOTA; blue - #active")
  fig.show()

In [28]:
#@title Proportion sota/active
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

def min_max(x):
    a = x.copy(deep=True)
    a["count"] = (a["count"]) / (a["count"].max())
    a = a.set_index(["year"])
    return a
    
def compose(df):
    # set to arbitrary result for now

    type0 = df[df["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type1 = df[df["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
    type3 = df[df["result"] == 3][["year", "count"]].sort_values(by="year").set_index("year")
    active = type0.subtract(type3, fill_value=0).cumsum()

    sota_prop = type1.divide(active, fill_value=0)

    print(type0.cumsum())
    print(type3.cumsum())
    print(active)
    print(type1)

    return sota_prop.reset_index()



for tlc, piktogram in all_res.groupby("top_level_class_label"):

  # piktogram = piktogram[piktogram["# results > 3"] == True]

  """
  grouping = pd.read_csv("trajectory_grouping_ITO_00141.csv")
  grouping.rename(columns={"Class_Label": "task_label"}, inplace=True)

  piktogram = piktogram.merge(grouping, on="task_label", how="left")
  piktogram["task_label"] = piktogram["Suggested_label"]+": "+ piktogram["task_label"]
  piktogram = piktogram.dropna()
  """

  piktogram = piktogram.sort_values(by=["task_label", "year", "result"])

  piktogram["result_label"] = piktogram["result"].map({0: 'New benchmarks', 1: 'New sota', 2: 'New entries but not sota', 3: 'Disbanded benchmarks'})

  piktogram["count"] = 1
  piktogram=piktogram.groupby(["year", "result"])["count"].sum().reset_index()

  # piktogram=piktogram.groupby(["task_label", "result"])["year", "count"].apply(cumsum).reset_index()


  # Compose

  type0 = piktogram[piktogram["result"] == 0][["year", "count"]].sort_values(by="year").set_index("year")
  type1 = piktogram[piktogram["result"] == 1][["year", "count"]].sort_values(by="year").set_index("year")
  type3 = piktogram[piktogram["result"] == 3][["year", "count"]].sort_values(by="year").set_index("year")

  active = type0.subtract(type3, fill_value=0).cumsum()
  active = active.reindex(range(active.index.min(),active.index.max() + 1)).ffill()

  active_norm = min_max(active.reset_index()).reset_index()

  sota_prop = type1.divide(active, fill_value=0).reset_index()

  """
  print(type0.cumsum())
  print(type3.cumsum())
  print(active)
  print(type1)
  print(sota_prop)
  """

  fig = go.Figure()

  fig.add_trace(go.Scatter(
                          x=[sota_prop["year"].min(), sota_prop["year"].max()], y=np.full(2, 0),
                          mode='lines',
                          line_color='white', showlegend=False))

  fig.add_trace(go.Scatter(x=sota_prop["year"], y=sota_prop["count"],
                  mode='lines', fill='tonexty', line_shape="linear"))

  fig.add_trace(go.Scatter(
                          x=[active_norm["year"].min(), active_norm["year"].max()], y=np.full(2, 0),
                          mode='lines',
                          line_color='white', showlegend=False))

  fig.add_trace(go.Scatter(x=active_norm["year"], y=active_norm["count"],
                  mode='lines', fill='tonexty', line_shape="linear"))
      
  #fig.update_traces(orientation='h', side='positive', width=3, points=False)
  # yaxis=dict(showticklabels=False)
  fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False, height=1000, showlegend=False, 
      title=f"{tlc}: red - Proportion #SOTA/#active; blue - Proportion #active/#max active")
  fig.show()